** Car Pooling App ** 

This app runs in Streamlit and manages rides for kids in sports matches.

It was initially projected for the AAC Handball teams, of different ages, 
to ease the assignment of kids to available cars, as well as serve as a match calendar
for the different teams.

Its a multi page app, running with Python under Streamlit and an external PostgreSQL database.
It manages athletes, age-group teams, matches, and car assignments per match.

############################################################################################################

In [ ]:
Example of Home.py

In [ ]:
import streamlit as st
import importlib
from db_connection import connect_db, initialize_db  # Use correct function names

# Establish a database connection at the start
conn = connect_db()

# Check if the connection was established successfully
if conn:
    initialize_db(conn)  # Initialize the database only if connection is successful

# Custom CSS for button formatting, logo size, and hiding Streamlit menu
st.markdown("""
    <style>
    div.stButton > button {
        display: block;
        width: 100%;
        border: none;
        background-color: #333333;
        color: white;
        font-size: 16px;
        padding: 12px;
        text-align: left;
        margin-bottom: 5px;
        border-radius: 8px;
    }
    div.stButton > button:hover {
        background-color: #444444;
        color: #00FFFF;  /* Cyan blue on hover */
    }
    .selected-button {
        background-color: #444444;
        color: white;
        font-weight: bold;
    }
    #MainMenu {visibility: hidden;} /* Hide Streamlit's default menu */
    .block-container {
        padding-top: 0 !important;
    }
    </style>
    """, unsafe_allow_html=True)

# Define the pages and which roles can access them
pages = {
    "Home": ["general", "admin"],  # Home is accessible before login only
    "Próximos Jogos": ["general", "admin"],
    "Atletas": ["admin"],
    "Jogos": ["admin"],
    "Escalões": ["admin"],
    "Jogos Passados": ["general", "admin"],
    "Informações da App": ["general", "admin"]
}

# Map menu page names to corresponding file names
page_files = {
    "Próximos Jogos": "1_Próximos_Jogos",
    "Atletas": "2_Atletas",
    "Jogos": "3_Jogos",
    "Escalões": "4_Escalões",
    "Jogos Passados": "5_Jogos_Passados",
    "Informações da App": "6_Informação_da_App"
}

# Authentication logic
def authenticate():
    if 'authenticated' not in st.session_state:
        st.session_state.authenticated = False
    if 'user_role' not in st.session_state:
        st.session_state.user_role = None

    # If authenticated, skip login
    if st.session_state.authenticated:
        return

    # Authentication form
    with st.form(key='login_form'):
        password = st.text_input('Insira Password:', type='password')
        login_button = st.form_submit_button('Login')

        if login_button:
            if password == 'aac':
                st.session_state.authenticated = True
                st.session_state.user_role = 'general'
                st.success('Password Correta!')
                st.session_state.selected_page = "Próximos Jogos"  # Automatically select Próximos Jogos
                st.rerun()  # Automatically redirect after login
            elif password == 'aac_admin':
                st.session_state.authenticated = True
                st.session_state.user_role = 'admin'
                st.success('Password de Admin Correta!')
                st.session_state.selected_page = "Próximos Jogos"  # Automatically select Próximos Jogos
                st.rerun()  # Automatically redirect after login
            else:
                st.error('Password Incorreta.')

# Display the logo on the top of the page (reintroducing the logo)
st.markdown("<br><br>", unsafe_allow_html=True)  # Adds some vertical space before the logo
st.image("logo_aac.png", width=100)  # Adjusted logo size

# Call the authenticate function
authenticate()

# Ensure `selected_page` is initialized
if 'selected_page' not in st.session_state:
    st.session_state.selected_page = "Home"  # Default to Home

# Hide the "Home" page for logged-in users and automatically redirect to Próximos Jogos
if st.session_state.authenticated:
    allowed_pages = [page for page, roles in pages.items() if st.session_state.user_role in roles and page != "Home"]
else:
    allowed_pages = [page for page, roles in pages.items() if st.session_state.user_role in roles]

# Sidebar button-based navigation
st.sidebar.title("Menu")  # Changed the sidebar title to "Menu"

# Create button-based navigation in the sidebar
for page in allowed_pages:
    if st.sidebar.button(page):
        st.session_state.selected_page = page
        st.rerun()  # Make sure we refresh the page when a button is clicked

# Dynamically import and load the selected page content
if st.session_state.authenticated and st.session_state.selected_page != "Home":
    try:
        module_name = page_files.get(st.session_state.selected_page, None)
        if module_name:
            module = importlib.import_module(module_name)
            module.show_page()  # No need to pass connection; individual pages handle that
        else:
            st.write("No content available for this page.")
    except Exception as e:
        st.write(f"Error loading page content: {e}")
else:
    # Don't show anything when the user is not logged in
    pass

# Add a logout button only if the user is authenticated
if st.session_state.authenticated and st.sidebar.button('Logout'):
    st.session_state.authenticated = False
    st.session_state.user_role = None
    st.rerun()  # Reload the app to show the login form again


############################################################################################################

Some examples of the app screenshot on desktop.

<img src="1.png" alt="BL1" width="600"/>
<br><br>
<img src="2.png" alt="PL1" width="600"/>